<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week15/human_activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Activity Recognition using 2D-Pose

In this practical, we will be using data from the following Github repository to train our model to recognize human activities through "motion capture". 

https://github.com/stuarteiffert/RNN-for-Human-Activity-Recognition-using-2D-Pose-Input

The dataset is also available for download from Polymall, and also from AWS. GO ahead to download the file from Polymall and take a look at its contents.



## Section 1 - Import Libraries and Setup Folders

Run the following cell, as is, to import all necessary libraries and set up folders.

In [ ]:
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/it3103/2D-Pose-Data.zip
!unzip 2D-Pose-Data.zip




Then, run the following cell to import the necesary libraries that we need to use.


In [ ]:
import pandas as pd
import os
import numpy as np
import math
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Input, Bidirectional, Dropout, LSTM, TimeDistributed, Flatten


## Section 2 - Declare Functions for Processing and Normalizing Data

### Remapping of Joint Coordinates 

The dataset that we are using to train our model uses the CMU OpenPose format to the human poses. 

The CMU OpenPose format stores the coordinates of the joints in the following order:
0. Nose X, Y
1. Neck X, Y
2. Right Shoulder X, Y
3. Right Elbow X, Y
4. Right Wrist X, Y
5. Left Shoulder X, Y
6. Left Elbow X, Y
7. Left Wrist X, Y
8. Right Hip X, Y
9. Right Knee X, Y
10. Right Ankle X, Y
11. Left Hip X, Y
12. Left Knee X, Y
13. Left Ankle X, Y
14. Right Eye X, Y
15. Left Eye X, Y
16. Right Ear X, Y
17. Left Ear X, Y


Let's run the following cell with an empty placeholder function that does no processing to the skeleton keypoints.


In [ ]:

# Process OpenPose's Joints
#
def process_joints(x):

    # No processing yet.
    return x



## Section 3 - Declare Functions to Perform Procesing

Update the following codes:

1. In the first part, call the process_joints on the row variable and append the results into the x list. That can be achieved by doing this:

   ```
      x.append(process_joints(row))
   ```

2. Convert the input from the row into a one-hot vector. This can be achieved using the following codes:

   ```
        one_hot = [0] * num_of_classes
        one_hot[row[0] - 1] = 1
        y.append(one_hot)
   ```

In [ ]:
df = pd.read_csv('2D-Pose-Data/X_train.txt', header=None)

In [ ]:
df.head()

In [ ]:
label_df = pd.read_csv('2D-Pose-Data/Y_train.txt', header=None)

In [ ]:
label_df[0].value_counts()

In [ ]:
count = 0

for row in df.itertuples(index=False):
    print(row)
    count += 1
    if count == 4:
        break

In [ ]:
### sequence length is 32 time steps, each timestep has 36 features (x,y) of 18 joints

# This function processes the x input data.
#
def load_x(infilepath):
    df = pd.read_csv(infilepath, header=None)
    
    print ("Processing " + infilepath)
    x = []
    for row in tqdm(df.itertuples(index=False)):

        # TODO:
        # Load each row from the CSV file and call the 
        # process_joints function.
        #
        x.append(process_joints(row))

    x = np.array(x)
    x = np.reshape(x, (int(x.shape[0] / 32), 32, 36))
    
    print ("Done.")
    return x
    

# This function processes the y labels.
#
def load_y_into_one_hot(infilepath, num_of_classes):
    df = pd.read_csv(infilepath, header=None)
    
    print ("Processing " + infilepath)
    
    y = []
    index = 0
    for row in tqdm(df.itertuples(index=False)):

        # TODO: 
        # Convert each row in the y file into a one-hot 
        # vector.
        #
        one_hot = [0] * num_of_classes
        one_hot[row[0] - 1] = 1
        y.append(one_hot)

        index+=1
            
    y = np.array(y)
    
    print ("Done.")  
    return y




## Section 4 - Load and Process the Data

Modify the codes into the cell below to load the x and y data into the variables x_train, x_test, y_train, y_test accordingly.

```
# Load our X input data
x_train = load_x("2D-Pose-Data/X_train.txt")
x_test = load_x("2D-Pose-Data/X_test.txt")

# Load our Y classification data
y_train = load_y_into_one_hot("2D-Pose-Data/Y_train.txt", 6)
y_test = load_y_into_one_hot("2D-Pose-Data/Y_test.txt", 6)
```

In the meantime, it will be a good idea to study CSV data to understand how the coordinates are saved from the training data. Observe how some X, Y coordinates are zero, and think about how this can affect processing or affect accuracy.


In [ ]:
# TODO:
# Load our X input data
#
#...#
X_train = load_x("2D-Pose-Data/X_train.txt")
X_test = load_x("2D-Pose-Data/X_test.txt")

# TODO:
# Load our Y classification data
#
#...#
y_train = load_y_into_one_hot("2D-Pose-Data/Y_train.txt", 6)
y_test = load_y_into_one_hot("2D-Pose-Data/Y_test.txt", 6)


In [ ]:
X_train.shape

## Section 5 - Visualize Our Dataset

Unlike images or structured data, these set of data can't be displayed as they are since they will not make any sense. 

Try running the first cell below to display the inputs like intensities of pixels in an image. 

The following is what you can use to display a 2D (32 x 36 "pixels") image of the first sample.

```
sample = 0
plt.imshow(X_train[sample])
```

In [ ]:
# TODO:
# Each sample is a two-dimension array of (32 frames by 36 x- or y-coordinates)
#...#
sample = 0
plt.imshow(X_train[sample])


Then, run the following cell below to treat each frame as time on the x-axis, and each of the 36 numbers represented to x- or y-coordinate of the joint on the screen as individual series.  

The code to do so looks like the following:

```
sample = 0
plt.plot(X_train[sample])
```

In [ ]:
X_train[0].shape

In [ ]:
# TODO:
# Plot each individual frame along the x (time) axis, and
# treat each line as a series.
#...#
sample = 0
plt.plot(X_train[sample])

Both the visualizations above provides some visual clue as to how the different joints move over time, but they are still difficult to imagine and visualize.

Update the following cell below to do a scatter plot of the X and Y coordianates of the various joints and animating them so that we can see their movements over time.

To do so, you must:

1. Set the data from your x_train to the scatter plot.

   ```
    graph_x = x_train[sample][frame][0::2]
    graph_y = x_train[sample][frame][1::2]
    line.set_data(graph_x, graph_y)
    return line,
   ```

2. Set the X- and Y-axis limits to 0-800 and 600-0 respectively this way:

    ```
    ax.set_xlim(0, 800)
    ax.set_ylim(600, 0)
    ```

3. Call the matplotlib's animation library to animate your points:

    ```
    anim = animation.FuncAnimation(fig, animate_pose, 32,  interval=50, blit=True)
    rc('animation', html='jshtml')
    anim
```

NOTE: These are the various types of actions captured in the dataset:
JUMPING, JUMPING_JACKS, BOXING, WAVING_2HANDS, WAVING_1HAND", "CLAPPING_HANDS"

In [ ]:
sample = 20000

# This function returns a set of data for every frame that is
# called from the animation.FuncAnimation below.
#
def animate_pose(frame):
    # TODO:
    # Retrieve the even number values as X-coordinates
    # and the odd number values as Y-coordinates
    #
    # Once you have these 2 sets of values, you can
    # pass them into the line.set_data to get matplotlib
    # to draw a scatter plot 
    #
    #...#
    graph_x = X_train[sample][frame][0::2]
    graph_y = X_train[sample][frame][1::2]
    line.set_data(graph_x, graph_y)
    return line,

fig, ax = plt.subplots()
plt.close()

# TODO:
# Set the x-limit and the y-limit of the animated scatter-plot chart
# Use x-limit = (0, 800), y-limit = (600, 0) for the original raw coordinates
# Use x-limit = (-4, 4), y-limit = (-3, 3) for the processed coordinates
#...#

# ax.set_xlim(0, 800)
# ax.set_ylim(600, 0)
ax.set_xlim(-4, 4)
ax.set_ylim(3, -3)


line, = ax.plot([], [], 'o', color='black');

# TODO:
# Trigger the matplotlib's animation function and pass in
# the animate_pose function above. Additionally, write the necessary
# codes to run the animation within colab.
#
# NOTE: The following codes may not work when running within Jupyter notebook
#
#...#
anim = animation.FuncAnimation(fig, animate_pose, 32,  interval=50, blit=True)
rc('animation', html='jshtml')
anim

## Section 6 - Define and Train Your Model

Try to define your Keras sequential model using any Keras layers. You should have minimally:

1. At least 1 RNN, or GRU, or LSTM Layer
2. At least 1 Dense Layer (for the softmax classification)
3. The input shape should be (32, 36)

Try to experiment and find a suitable model. You can decide if you want to use Bidirectional models, and add any Dropout layers if it helps to improve your model performance. Share your model with the class together with the validation accuracy!

A typical model can be designed this way:

```
    model.add(LSTM(128, input_shape=(32, 36)))
    model.add(Dropout(0.2))

    model.add(Dense(6, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
```

This is part of model selection and evaluation in any machine learning project. 

You may find that a good validation accuracy for you model may hover near about 85-90%. 

In [ ]:
# Create our LSTM model here
#
def create_model():

    # Use Keras to create a Sequential model here with any layers that 
    # you'd like.
    #
    model = Sequential()

    # TODO:
    # Apply your Keras knowledge and create your layers
    #
    #...#
    model.add(LSTM(128, input_shape=(32, 36)))
    model.add(Dropout(0.2))

    model.add(Dense(6, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model



# Create our model
#
model = create_model()


# Create the training folder
#
training_session_id = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
training_session_folder = '/train_%s' % (training_session_id)
os.makedirs(training_session_folder, exist_ok=True)

# Configure the checkpoint and stop point.
# This allows the training to save the best models and also stop the
# training early if it detects that there are no improvements after
# a long time.
#
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath=training_session_folder + '/model.{epoch:04d}-val_acc-{val_accuracy:4.2f}-loss-{val_loss:4.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]


# Train our model
#
history = model.fit(x=X_train, y=y_train, batch_size=200, epochs=20, verbose=1, callbacks=callbacks_list, validation_data=(X_test, y_test))

## Section 7 - Scale / Translation Normalization

So far, we have not talked about how we can normalize our skeletal keypoints so that the pose data is scale / translation invariant. This means that regardless of how far the person is from the camera, or when the person moves left or right or up or down, the coordinates of all joint positions should always be relative to a fixed frame of reference.

To take care of translation (left / right / up / down) invariance, we are shift all points together so that neck point is always placed at (0, 0). 

To take care of scale invariance, we estimate the torso height (which is either the length of the neck point to either hip, or the width of the shoulders). We then divide all joint coordinates by the torso height.

To do so, update the following the process_joints function to include code to normalize the skeleton key points as described above:

1. ref = P[1] or the midpoint of P[2], P[5]
2. reflength = length(ref to P[8]) or length(ref to P[11]) 
3. Compute 
   - P[i].x = (P[i].x - ref.x) / reflength
   - P[i].y = (P[i].y - ref.y) / reflength

```

# Declare a function that can compute length between two points
#   (x1,y1) - (x2,y2)
#
def compute_length(x1, y1, x2, y2):
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Process OpenPose's Joints
# NOTE: The "x" parameter consists of an array of consecutive x and y values 
# within the same array.
#
# x = [p0.x, p0.y, p1.x, p1.y, p2.x, p2.y, ..., p17.x, p17.y]
#     (a total of 36 values)
# 
def process_joints(x):

    r = [0] * 36

    # Initialize some values for the reference length and the reference point.
    #
    refx = 0
    refy = 0
    reflength = 1

    # Step 1: Let's find the reference point (neck)
    #
    if x[2] != 0 or x[3] != 0:         
        refx = x[2]                # use the neck X, Y
        refy = x[3]
    elif (x[4] != 0 or x[5] != 0) and (x[10] != 0 or x[11] != 0):
        refx = (x[4] + x[10]) / 2  # estimate the neck X, Y from the mid point
        refy = (x[5] + x[11]) / 2  # of the left/right shoulder
    
    # Step 2: Let's first estimate the torso length.
    #
    if x[16] != 0 and x[17] != 0:             
        reflength = compute_length(x[16], x[17], refx, refy)   # neck to right hip
    elif x[22] != 0 and x[23] != 0:
        reflength = compute_length(x[22], x[23], refx, refy)   # neck to left hip

    # Step 3:
    # Perform the translation and the scaling.
    #
    for i in range(0, 18):
        r[i*2] = (x[i*2] - refx) / reflength
        r[i*2 + 1] = (x[i*2 + 1] - refy) / reflength
    
    # Return the re-mapped and normalized result
    #
    return r
```

In [ ]:
# TODO:
# Declare a function that can compute length between two points
#   (x1,y1) - (x2,y2)
#
def compute_length(x1, y1, x2, y2):
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Process OpenPose's Joints
# NOTE: The "x" parameter consists of an array of consecutive x and y values 
# within the same array.
#
# x = [p0.x, p0.y, p1.x, p1.y, p2.x, p2.y, ..., p17.x, p17.y]
#     (a total of 36 values)
# 
def process_joints(x):

    r = [0] * 36

    # TODO:
    # Initialize some values for the reference length and the reference point.
    #
    refx = 0
    refy = 0
    reflength = 1

    # TODO:
    # Step 1: Let's find the reference point (neck)
    #
    if x[2] != 0 or x[3] != 0:         
        refx = x[2]                # use the neck X, Y
        refy = x[3]
    elif (x[4] != 0 or x[5] != 0) and (x[10] != 0 or x[11] != 0):
        refx = (x[4] + x[10]) / 2  # estimate the neck X, Y from the mid point
        refy = (x[5] + x[11]) / 2  # of the left/right shoulder
    
    # TODO:
    # Step 2: Let's first estimate the torso length.
    #
    if x[16] != 0 and x[17] != 0:             
        reflength = compute_length(x[16], x[17], refx, refy)   # neck to right hip
    elif x[22] != 0 and x[23] != 0:
        reflength = compute_length(x[22], x[23], refx, refy)   # neck to left hip

    # TODO:
    # Step 3:
    # Perform the translation and the scaling.
    #
    for i in range(0, 18):
        r[i*2] = (x[i*2] - refx) / reflength
        r[i*2 + 1] = (x[i*2 + 1] - refy) / reflength
    
    # Return the re-mapped and normalized result
    #
    return r



## Section 8 - Re-Load the Data and Re-Train Your Model Again

At this juncture, re-run the cells under the following sections again without changing the model or training parameters:

- Section 4 - Processing the Data
- Section 6 - Define and Train Your Model

See how the model's training and validation accuracy has changed after normalizating the data.

**Discuss:**

1. Why did the normalization cause a change in the training / validation performance?
2. What do you think you can do to improve performance instead of normalizating the data?
3. Other than scaling / translating the points, what other kinds of normalization can you do?
4. What are the downsides to normalizing the data?

## Section 9 - Evaluate the Training History and Performance



In [ ]:
#------------------------------------------------------------------------------------------

import matplotlib.pyplot as plt

# This function is used to display the history the train/test accuracy/loss
# of the Keras training.
#
#   history - Pass in the history returned from the model.fit(...) method.
#
def display_training_loss_and_accuracy(history):
    
    plt.figure(figsize=(20,4))
    
    # summarize history for accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    # summarize history for loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()    

display_training_loss_and_accuracy(history)




In [ ]:
labels = ["JUMPING", "JUMPING_JACKS", "BOXING", "WAVING_2HANDS", "WAVING_1HAND", "CLAPPING_HANDS"]

In [ ]:
#------------------------------------------------------------------------------------------

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
 
def display_classification_confusion_matrix(keras_model, x_train, y_train, x_test, y_test, labels):
    
    '''
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    '''
    pred_y_train = []
    pred_y_test = []

    print (x_train.shape)
    pred_y_train = keras_model.predict(x_train)

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    pred_y_train = np.array(pred_y_train)

    pred_y_test = keras_model.predict(x_test)

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    pred_y_test = np.array(pred_y_test)

    #test_generator.on_epoch_end()

    plt.figure(figsize=(20,6))  

    labels = np.array(labels)

    # Convert the target labels into the categorical index
    #
    y_train_index = y_train.argmax(axis=1)
    pred_y_train_index = pred_y_train.argmax(axis=1)
    y_test_index = y_test.argmax(axis=1)
    pred_y_test_index = pred_y_test.argmax(axis=1)
    
    # Print the first Confusion Matrix for the training data
    #
    cm = confusion_matrix(y_train_index, pred_y_train_index)

    cm_df = pd.DataFrame(cm, labels, labels)          
    plt.subplot(1, 2, 1)
    plt.title('Confusion Matrix (Train Data)')
    sns.heatmap(cm_df, annot=True)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')        
    
    # Print the second Confusion Matrix for the test data
    #    
    cm = confusion_matrix(y_test_index, pred_y_test_index)
    
    cm_df = pd.DataFrame(cm, labels, labels)          
    plt.subplot(1, 2, 2)
    plt.title('Confusion Matrix (Test Data)')
    sns.heatmap(cm_df, annot=True)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')        
    
    plt.show()

    # Finally display the classification reports
    #
    print ("Train Data:")
    print ("--------------------------------------------------------")
    print(classification_report(y_train_index, pred_y_train_index, target_names=labels))
    print ("")
    print ("Test Data:")
    print ("--------------------------------------------------------")
    print(classification_report(y_test_index, pred_y_test_index, target_names=labels))
    

# Exclude the O tags from the confusion matrix.
#
display_classification_confusion_matrix(model, X_train, y_train, X_test, y_test, labels)

## Section 9 - Save and Download Model

Run the following cell to save your model. 





In [ ]:
model.save("model.savedmodel")



Run the following the zip the "model.savedmodel" folder into a single zip file.

Download that zip file from Colab once you are done! We will be using this for the next practical exercise.

In [ ]:
!zip model.savedmodel.zip -r model.savedmodel

In [ ]:
model = keras.models.load_model('model.savedmodel')

In [ ]:
model.summary()

In [ ]:
labels

In [ ]:
sample_index = 2000
sample = X_test[sample_index]
label = y_test[sample_index]


In [ ]:
sample = np.expand_dims(sample, axis=0)

In [ ]:
pred = model(sample)

In [ ]:
print('actual = {}'.format(labels[np.argmax(label)]))
print('predicted = {}'.format(labels[np.argmax(pred)]))